In [1]:
%matplotlib osx

# Drawing images on the Sag River

* Sag: 19060402
* Headwater 12: 190604020404


In [2]:
import os,sys
import numpy as np
from matplotlib import pyplot as plt
import shapely
import logging
import pysheds.grid

import workflow
import workflow.source_list
import workflow.ui
import workflow.colors
import workflow.condition
import workflow.bin_utils
import workflow.plot

workflow.ui.setup_logging(1,None)

/Users/uec/codes/anaconda/3/envs/ats_meshing_20200525/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/Users/uec/codes/anaconda/3/envs/ats_meshing_20200525/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)
/Users/uec/codes/anaconda/3/envs/ats_meshing_20200525/lib/python3.7/site-packages/pyproj/crs/crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is depreca

## Sources and data collection

A wide range of data sources are available; here we use the defaults except for using NHD "Highest Res" dataset (best available in AK)

In [3]:
# set up a dictionary of source objects
sources = workflow.source_list.get_default_sources()
workflow.source_list.log_sources(sources)

2020-07-31 15:42:58,372 - root - INFO: Using sources:
2020-07-31 15:42:58,374 - root - INFO: --------------
2020-07-31 15:42:58,376 - root - INFO: HUC: National Watershed Boundary Dataset (WBD)
2020-07-31 15:42:58,378 - root - INFO: hydrography: National Hydrography Dataset (NHD)
2020-07-31 15:42:58,379 - root - INFO: DEM: National Elevation Dataset (NED)
2020-07-31 15:42:58,380 - root - INFO: soil type: National Resources Conservation Service Soil Survey (NRCS Soils)
2020-07-31 15:42:58,381 - root - INFO: land cover: National Land Cover Database (NLCD) Layer: NLCD_2016_Land_Cover_L48
2020-07-31 15:42:58,383 - root - INFO: soil thickness: None
2020-07-31 15:42:58,384 - root - INFO: meteorology: DayMet 1km


In [4]:
# specify the input shapefile and a hint as to what HUC it is in.
huc = '190604020404'
logging.info("")
logging.info("Meshing HUC: {}".format(huc))
logging.info("="*30)

# get the shape and crs of the shape
crs, hucs = workflow.get_split_form_hucs(sources['HUC'], huc, 12, crs=workflow.crs.default_alaska_crs())

2020-07-31 15:43:12,934 - root - INFO: 
2020-07-31 15:43:12,935 - root - INFO: Meshing HUC: 190604020404
2020-07-31 15:43:12,937 - root - INFO: ==============================
2020-07-31 15:43:12,942 - root - INFO: 
2020-07-31 15:43:12,945 - root - INFO: Preprocessing HUC
2020-07-31 15:43:12,947 - root - INFO: ------------------------------
2020-07-31 15:43:12,948 - root - INFO: Loading level 12 HUCs in 190604020404.
2020-07-31 15:43:12,953 - root - INFO: Using HUC file "/Users/uec/research/water/data/watershed-workflow/data-master/hydrography/WBD_19_GDB/WBD_19.gdb"
2020-07-31 15:43:16,006 - root - INFO:   found 1 HUCs.
2020-07-31 15:43:16,007 - root - INFO:   -- 190604020404


In [5]:
# plot the shape, does it look right?
fig, ax = workflow.plot.get_ax(crs)
workflow.plot.shply([hucs.exterior(),], crs, ax=ax)


In [17]:
# download/collect the river network within that shape's bounds
_, reaches = workflow.get_reaches(sources['hydrography'], huc, 
                                  hucs.exterior().bounds, crs)

# simplify and prune rivers not IN the shape, constructing a tree-like data structure
# for the river network
rivers = workflow.simplify_and_prune(hucs, reaches, simplify=20, cut_intersections=True)



2020-07-31 15:50:42,977 - root - INFO: 
2020-07-31 15:50:42,979 - root - INFO: Preprocessing Hydrography
2020-07-31 15:50:42,981 - root - INFO: ------------------------------
2020-07-31 15:50:42,982 - root - INFO: Loading streams in HUC 190604020404
2020-07-31 15:50:42,983 - root - INFO:          and/or bounds (195504.56252458727, 2030373.258729276, 216355.9374925703, 2048392.8056229195)
2020-07-31 15:50:42,986 - root - INFO: Using Hydrography file "/Users/uec/research/water/data/watershed-workflow/data-master/hydrography/NHD_H_19060402_GDB/NHD_H_19060402.gdb"
2020-07-31 15:50:43,289 - root - INFO:   found 545 reaches
2020-07-31 15:51:50,605 - root - INFO: 
2020-07-31 15:51:50,606 - root - INFO: Simplifying and pruning
2020-07-31 15:51:50,607 - root - INFO: ------------------------------
2020-07-31 15:51:50,608 - root - INFO: Filtering rivers outside of the HUC space
2020-07-31 15:51:50,609 - root - INFO:   ...filtering
2020-07-31 15:51:50,714 - root - INFO:   filtered from 471 to 141 

In [18]:
# braided network -- we must figure out how to simplify 
reaches = workflow.hydrography.filter_rivers_to_shape(hucs.exterior(), reaches, 1)
rivers = workflow.hydrography.make_global_tree(reaches)
rivers = [r for r in rivers if len(r) > 2]
print(len(rivers))

2020-07-31 16:21:32,075 - root - INFO:   ...filtering


1


In [24]:
# get a raster on that watershed -- getting raster in native crs
dem_profile, dem = workflow.get_raster_on_shape(sources['DEM'], hucs.exterior(), crs, crs, 
                                mask=True, nodata=np.nan)
print(dem_profile)

2020-07-31 16:23:15,414 - root - INFO: 
2020-07-31 16:23:15,415 - root - INFO: Preprocessing Raster
2020-07-31 16:23:15,416 - root - INFO: ------------------------------
2020-07-31 16:23:15,418 - root - INFO: collecting raster
2020-07-31 16:23:15,595 - root - INFO: Collecting DEMs to tile bounds: [-149.3337383807433, 68.15835009523076, -148.79217986384458, 68.33619896061079]
2020-07-31 16:23:15,596 - root - INFO:   Need:
2020-07-31 16:23:15,596 - root - INFO:     /Users/uec/research/water/data/watershed-workflow/data-master/dem/USGS_NED_1as_n69_w150.img
2020-07-31 16:23:15,597 - root - INFO:     /Users/uec/research/water/data/watershed-workflow/data-master/dem/USGS_NED_1as_n69_w149.img
2020-07-31 16:23:15,691 - root - INFO: Got raster of shape: (641, 1950)
2020-07-31 16:23:15,824 - root - INFO: Masking to shape
2020-07-31 16:23:15,841 - root - INFO: Raster bounds: (193937.88506934882, 2050090.5044407842, 217948.60719914178, 2028781.7188036991)


{'driver': 'HFA', 'dtype': 'float32', 'nodata': nan, 'width': 1644, 'height': 1459, 'count': 1, 'crs': CRS.from_epsg(3338), 'transform': Affine(14.605062122745112, 0.0, 193937.88506934882,
       0.0, -14.605062122745112, 2050090.5044407842), 'blockxsize': 64, 'blockysize': 64, 'tiled': True}


In [20]:
print(crs)

EPSG:3338


In [28]:
# plot the dem
fig, ax = workflow.plot.get_ax(crs, figsize=(10,10))
workflow.plot.dem(dem_profile, dem, ax=ax)
workflow.plot.dem(dem_profile, dem, ax=ax, cmap='prism', alpha=0.1)
workflow.plot.rivers(rivers, crs, color='white', ax=ax, linewidth=1)
workflow.plot.hucs(hucs, crs, 'r', ax=ax, linewidth=0.6)
ax.outline_patch.set_visible(False)
ax.background_patch.set_visible(False)
fig.savefig('sag_dem', dpi=300, transparent=True)
plt.show()

2020-07-31 16:24:55,381 - root - INFO: BOUNDS: (193937.88506934882, 2028781.7188036991, 217948.60719914178, 2050090.5044407842)
2020-07-31 16:24:55,402 - root - INFO: BOUNDS: (193937.88506934882, 2028781.7188036991, 217948.60719914178, 2050090.5044407842)
/Users/uec/codes/anaconda/3/envs/ats_meshing_20200525/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.
  import sys
/Users/uec/codes/anaconda/3/envs/ats_meshing_20200525/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: The background_patch property is deprecated. Use GeoAxes.patch instead.
  


In [ ]:
# limit the above plot to the HUC
dem_profile, dem = workflow.get_raster_on_shape(sources['DEM'], hucs.exterior(), crs, crs, mask=True, nodata=np.nan)

fig, ax = workflow.plot.get_ax(crs, figsize=(8,8))
workflow.plot.dem(dem_profile, dem, ax=ax)
workflow.plot.shply(reaches, crs, color='white', ax=ax)
workflow.plot.hucs(hucs, crs, 'r', ax=ax, linewidth=0.6)
plt.show()

In [ ]:
# yikes, that's a braided channel....

# how can we simplify this in ways that -- 
#   1, allows the specification of a delta channel length
#   2, collapses braided channels
#   3, removes segments by contributing area

rivers = workflow.hydrography.make_global_tree(reaches)
rivers = workflow.hydrography.filter_rivers_to_shape(hucs.exterior(), rivers, 1)
rivers = [r for r in rivers if len(r) > 2]
assert(len(rivers) == 1)
river = rivers[0]

In [ ]:
print([len(r) for r in rivers])

In [ ]:
# write shape, DEM to disk for Jon
import fiona, rasterio, collections

# write these subbasins to file for future use
schema = dict()
schema['geometry'] = 'Polygon'
schema['properties'] = collections.OrderedDict()
schema['properties']['name'] = 'str'

# east river
properties = collections.OrderedDict()
name = 'huc_{}'.format(huc)
properties['name'] = name
with fiona.open('./{}.shp'.format(name), 'w', 
                driver='ESRI Shapefile', crs=workflow.crs.to_fiona(crs), crs_wkt=workflow.crs.to_wkt(crs),
                schema=schema) as c:
    c.write({'geometry': shapely.geometry.mapping(hucs.exterior()),
             'properties': properties,
            })

rio_profile = dict(dem_profile).copy()
rio_profile.pop('blockxsize')
rio_profile.pop('blockysize')
rio_profile.pop('tiled')
rio_profile['nodata'] = np.float32(0)
rio_profile['driver'] = 'GTiff'
rio_profile['compress'] = 'lzw'

rio_dem = np.where(np.isnan(dem), rio_profile['nodata'], dem).astype(rio_profile['dtype'])

for k,v in rio_profile.items():
    print(k,v)

with rasterio.open('./{}.tif'.format(name), 'w', **rio_profile) as dst:
    dst.write(rio_dem,1)

        


In [ ]:
# can we read it?
dset = rasterio.open('./huc_190604020404.tif')
dat = dset.read(1)
plt.imshow(dat)



In [ ]:
# plot the resulting surface mesh
fig = plt.figure(figsize=figsize_3d)
ax = workflow.plot.get_ax('3d', fig, window=[0.0,0.2,1,0.8])
cax = fig.add_axes([0.23,0.18,0.58,0.03])

mp = ax.plot_trisurf(mesh_points3[:,0], mesh_points3[:,1], mesh_points3[:,2], 
                     triangles=mesh_tris, cmap='viridis', 
                     edgecolor=(0,0,0,.2), linewidth=0.5)
cb = fig.colorbar(mp, orientation="horizontal", cax=cax)

rivers_2 = [np.array(r.xy) for riv in rivers for r in riv]
rivers_e = [workflow.values_from_raster(r.transpose(), crs, dem, dem_profile) 
               for r in rivers_2]
rivers_l3 = [np.array([i[0], i[1], j]).transpose() 
               for i,j in zip(rivers_2, rivers_e)]
for r in rivers_l3:
    ax.plot(r[:,0]+1, r[:,1], r[:,2]+10, color='red', linewidth=3)

t = cax.set_title('elevation [m]')
ax.view_init(55,0)
ax.set_xticklabels(list())
ax.set_yticklabels(list())

fig.savefig('coweeta_dem_3d')

In [ ]:
# plot the resulting surface mesh
fig = plt.figure(figsize=figsize)
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 color='elevation', edgecolor='gray', linewidth=0.2)
cbar = fig.colorbar(mp, orientation="horizontal", pad=0.05)
workflow.plot.hucs(coweeta, crs, ax=ax, color='k', linewidth=1)
workflow.plot.rivers(rivers, crs, ax=ax, color='red', linewidth=1)
ax.set_aspect('equal', 'datalim')
ax.set_title('elevation [m]')
cbar.ax.set_title('elevation [m]')
fig.savefig('coweeta_dem')

In [ ]:
# construct the 2D mesh
m2 = workflow.extrude.Mesh2D(mesh_points3.copy(), list(mesh_tris))

In [ ]:
# hydrologically condition the mesh
workflow.condition.condition(m2)

# plot the change between the two meshes
diff = np.copy(mesh_points3)
diff[:,2] = m2.points[:,2] - mesh_points3[:,2] 
print("max diff = ", np.abs(diff[:,2]).max())
fig, ax = workflow.plot.get_ax(crs, figsize=figsize)
workflow.plot.triangulation(diff, m2.conn, crs, color='elevation', edgecolors='gray', 
                            linewidth=0.2, ax=ax)
ax.set_title('conditioned dz')
plt.show()

## Surface properties

Meshes interact with data to provide forcing, parameters, and more in the actual simulation.  Specifically, we need vegetation type on the surface to provide information about transpiration and subsurface structure to provide information about water retention curves, etc.

We'll start by downloading and collecting land cover from the NLCD dataset, and generate sets for each land cover type that cover the surface.  Likely these will be some combination of grass, deciduous forest, coniferous forest, and mixed.

In [ ]:
# download the NLCD raster
lc_profile, lc_raster = workflow.get_raster_on_shape(sources['land cover'], 
                                                     coweeta.exterior(), crs)

# resample the raster to the triangles
lc = workflow.values_from_raster(m2.centroids(), crs, lc_raster, lc_profile)

# what land cover types did we get?
logging.info('Found land cover dtypes: {}'.format(lc.dtype))
logging.info('Found land cover types: {}'.format(set(lc)))


In [ ]:
# plot the NLCD data

# -- get the NLCD colormap which uses official NLCD colors and labels
nlcd_indices, nlcd_cmap, nlcd_norm, nlcd_ticks, nlcd_labels = \
                workflow.colors.generate_nlcd_colormap(lc)

nlcd_labels_fw = []
for label in nlcd_labels:
    label_fw = label
    if len(label) > 15:
        if ' ' in label:
            lsplit = label.split()
            if len(lsplit) == 2:
                label_fw = '\n'.join(lsplit)
            elif len(lsplit) == 4:
                label_fw = '\n'.join([' '.join(lsplit[0:2]),
                                      ' '.join(lsplit[2:])])
            elif len(lsplit) == 3:
                if len(lsplit[0]) > len(lsplit[-1]):
                    label_fw = '\n'.join([lsplit[0],
                                          ' '.join(lsplit[1:])])
                else:
                    label_fw = '\n'.join([' '.join(lsplit[:-1]),
                                          lsplit[-1]])
    nlcd_labels_fw.append(label_fw)

fig = plt.figure(figsize=figsize)
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, color=lc,
                                 cmap=nlcd_cmap, norm=nlcd_norm, edgecolor='none', 
                                 linewidth=0.5)
cb = fig.colorbar(mp)
cb.set_ticks(nlcd_ticks)
cb.set_ticklabels(nlcd_labels_fw)
ax.set_title("land cover index")
fig.savefig('coweeta_nlcd')

## Subsurface properties

Get soil structure from SSURGO

In [ ]:
# download the NRCS soils data as shapes and project it onto the mesh
import workflow.sources.manager_nrcs
import matplotlib.cm

# -- download the shapes
target_bounds = coweeta.exterior().bounds
_, soil_survey = workflow.get_shapes(sources['soil type'], target_bounds, crs)

# -- log the bounds targetted and found
logging.info('target bounds: {}'.format(target_bounds))
logging.info('shape union bounds: {}'.format(
    shapely.ops.cascaded_union(soil_survey).bounds))

# -- determine the NRCS mukey for each soil unit; this uniquely identifies soil 
#    properties
soil_ids = np.array([shp.properties['id'] for shp in soil_survey], np.int32)

# -- color a raster by the polygons (this makes identifying a triangle's value much 
#    more efficient)
soil_color_raster, soil_color_profile, img_bounds = \
            workflow.color_raster_from_shapes(target_bounds, 10, soil_survey,
                                              soil_ids, crs)

# -- resample the raster to the triangles
soil_color = workflow.values_from_raster(m2.centroids(), crs, 
                                         soil_color_raster, soil_color_profile)

In [ ]:
# plot the soil data
fig = plt.figure(figsize=figsize)
ax = workflow.plot.get_ax(crs, fig)

mp = workflow.plot.triangulation(mesh_points3, mesh_tris, crs, ax=ax, 
                                 linewidth=0.5, color=soil_color, cmap='copper')
ax.set_title('soil type index')

fig.savefig('coweeta_soils')

## Mesh extrusion

Given the surface mesh and material IDs on both the surface and subsurface, we can extrude the surface mesh in the vertical to make a 3D mesh.

In [ ]:
# layer extrusion
# -- data structures needed for extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []
z = 0.0

# -- soil layer --
#  top 6 m
#  5 cm initial top cell
#  10 cells
#  expanding dz, growing with depth
ncells = 9
dz = 0.05
layer_dz = 4

def telescope_factor(ncells, dz, layer_dz):
    """Calculates a telescoping factor"""
    if ncells * dz > layer_dz:
        raise ValueError(("Cannot telescope {} cells of thickness at least {} "+
                          "and reach a layer of thickness {}"
                         ).format(ncells, dz, layer_dz))

    import scipy.optimize
    def seq(r):
        calc_layer_dz = dz * (1 - r**ncells)/(1-r)
        #print('tried: {} got: {}'.format(r, calc_layer_dz))
        return layer_dz - calc_layer_dz
    res = scipy.optimize.root_scalar(seq, x0=1.0001, x1=2)
    return res.root

tele = telescope_factor(ncells, dz, layer_dz)
logging.info("Got telescoping factor: {}".format(tele))
for i in range(ncells):
    layer_types.append('constant')
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(soil_color)
    z += dz
    dz *= tele
    
# one more 2m layer makes 6m
dz = 2.0
layer_types.append('constant')
layer_data.append(dz)
layer_ncells.append(1)
layer_mat_ids.append(soil_color)
z += dz

# -- geologic layer --
# keep going for 2m cells until we hit the bottom of
# the domain
layer_types.append("constant")
layer_data.append(40 - z) # depth of bottom of domain is 40 m
layer_ncells.append(int(round(layer_data[-1] / dz)))
layer_mat_ids.append(999*np.ones_like(soil_color))

# print the summary
workflow.extrude.Mesh3D.summarize_extrusion(layer_types, layer_data, 
                                            layer_ncells, layer_mat_ids)

In [ ]:
# extrude
m3 = workflow.extrude.Mesh3D.extruded_Mesh2D(m2, layer_types, layer_data, 
                                             layer_ncells, layer_mat_ids)

In [ ]:
# add back on land cover side sets
surf_ss = m3.side_sets[1]

for index, name in zip(nlcd_indices, nlcd_labels):
    where = np.where(lc == index)[0]
    ss = workflow.extrude.SideSet(name, int(index), 
                                  [surf_ss.elem_list[w] for w in where],
                                  [surf_ss.side_list[w] for w in where])        
    m3.side_sets.append(ss)

In [ ]:
# save to disk
try:
    os.remove('coweeta_basin.exo')
except FileNotFoundError:
    pass
m3.write_exodus('coweeta_basin.exo')